In [ ]:
%reset -fs

In [ ]:
import pandas as pd
import numpy as np
import time, os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from pymongo import MongoClient
import pandas as pd
import datetime

In [ ]:
client = MongoClient()

In [ ]:
db = client.nba_db
cursor = db.betting_data.find()
mongo_df = pd.DataFrame(list(cursor))

In [ ]:
today = datetime.datetime.today()
max_date = datetime.datetime.strptime(max(mongo_df.Date), '%Y-%m-%d')

def get_date_list(date):
    date_list = []
    today = datetime.datetime.today()
    max_date = datetime.datetime.strptime(max(mongo_df.Date), '%Y-%m-%d')
    delta = today - max_date
    date_list = [max_date - datetime.timedelta(days=x) for x in range(0, delta.days)]
    date_list = [d.strftime('%Y-%m-%d') for d in date_list]
    date_list.append((datetime.datetime.today() - datetime.timedelta(days=1)).strftime('%Y-%m-%d'))
    date_list.append(datetime.datetime.today().strftime('%Y-%m-%d'))
    date_list = set(date_list)
    date_list = list(date_list)
    return date_list
    
    
    

In [ ]:
date_list = get_date_list(today)


In [ ]:
chromedriver = "/Applications/chromedriver"
os.environ['webdriver.chrome.driver'] = chromedriver
driver = webdriver.Chrome(chromedriver)

In [ ]:
bet_search =  'https://www.bettingpros.com/nba/picks/prop-bets/bet/points/?date='

In [ ]:
def get_site_rb(date):
    bet_query = bet_search + date
    driver.get(bet_query)
    time.sleep(5)
    try:
        element = driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')
        action = ActionChains(driver)
        action.move_to_element(element).click().perform()
    except:
        pass
    get_data_rb()


In [ ]:
def get_data_rb():
    data_str = []
    data = []
    li = []
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    table = soup.find('table')
    date = soup.find('h1').text
    date = date.split(" ")[5]
    date.strip(')')
    date = date.replace('(',"").replace(')',"")
    house_accuracy = soup.find('div', class_='best-props__accuracy').text
    table_data = table.tbody.find_all('tr')
    for tr in table_data:
        data_str.append(tr.text.replace('\n', ' ').strip())
    for i in data_str:
        li = list(i.split('            '))
        data.append(li)
    for i in data:
        if len(i) == 7:
            i.append('TBD')
            i.append(date)
            i.append(house_accuracy)
        else:
            i.append(date)
            i.append(house_accuracy)
    global final_df
    final_df = pd.concat([final_df, pd.DataFrame(data)])
    time.sleep(1)

    try:
        page_progress = driver.find_element_by_xpath('//div[contains(@class, "table__pagination")]').text.strip("«").strip("»").strip()
        if int(page_progress[5]) == int(page_progress[10]):
            return final_df
        else:
            element = driver.find_element_by_xpath('/html/body/div[2]/main/div/div/div/div[4]/div/div/a[2]')
            action = ActionChains(driver)
            action.move_to_element(element).click().perform()
            time.sleep(3)
            get_data_rb()
    except:
        return final_df

In [ ]:
final_df = pd.DataFrame()

In [ ]:
for i in date_list:
    get_site_rb(i)

In [ ]:
driver.close()

In [ ]:
#Clean up below
final_df.columns=['Name', 'Team', 'Opp', 'Type', 'Line', 'Pick', 'Moneyline', 'Result', 'Date', 'Accuracy']

In [ ]:
final_df.reset_index(drop=True, inplace=True)

In [ ]:
player_name = final_df.Name.str.split(" ", expand = True)
final_df['Name'] = player_name[2] + ' ' + player_name[4]

In [ ]:
team = final_df.Team.str.split(" ", expand = True)
final_df['Team'] = team[0]
final_df['Position'] = team[2]

In [ ]:
opp = final_df.Opp.str.split(" ", expand = True)
final_df.Opp = opp[1]
final_df['Home/Away'] = opp[0]

In [ ]:
bet_info = final_df.Line.str.split(" ", expand = True)
final_df['Line'] = bet_info[7]
final_df['Projection'] = bet_info[15]
final_df['Diff'] = bet_info[9]
final_df['Moneyline'] = bet_info[13]
final_df['over_under'] = bet_info[16]
final_df['over_under'] = final_df['over_under'].fillna('o')

In [ ]:
final_df = final_df[['Date', 'Name', 'Team', 'Position', 'Opp','Home/Away', 'Line', 'Moneyline', 'Projection', 'Diff', 'Pick', 'Result', 'Accuracy', 'over_under']]

In [ ]:
final_df = final_df.copy()

In [ ]:
final_df['Projection'] = final_df['Projection'].map(lambda x: x.rstrip('ou'))

In [ ]:
accuracies = final_df.Accuracy.str.split(" ", expand = True)

In [ ]:
final_df['seven_day_accuracy'] = accuracies[8]

In [ ]:
final_df.seven_day_accuracy = final_df.seven_day_accuracy.replace('', 50, regex=True)

In [ ]:
final_df['seven_day_accuracy'] = final_df['seven_day_accuracy'].map(lambda x: x.lstrip('(').rstrip('%)'))

In [ ]:
final_df['seven_day_accuracy'] = pd.to_numeric(final_df['seven_day_accuracy'])

In [ ]:
final_df['seven_day_accuracy'] = final_df['seven_day_accuracy'].map(lambda x: x/100)

In [ ]:
final_df.drop(columns=['Accuracy'], inplace=True)

In [ ]:
final_df['Date'] = pd.to_datetime(final_df['Date'])

In [ ]:
# Final form betting df, additional cleaning and saving to csv below
sorted_df = final_df.sort_values(by=['Name', 'Date'])
sorted_df.reset_index(inplace=True, drop=True)

In [ ]:
sorted_df['Line'] = pd.to_numeric(sorted_df.Line)

In [ ]:
sorted_df = sorted_df.replace('EVEN', '+100')

In [ ]:
sorted_df['Moneyline'] = pd.to_numeric(sorted_df.Moneyline)

In [ ]:
sorted_df['Result'] = sorted_df.Result.apply(lambda x: str(x))

In [ ]:
results = sorted_df.Result.str.split(' ', expand=True);

In [ ]:
try:
    sorted_df.Result = results[3]
except:
    sorted_df.Result = results[0]

In [ ]:
sorted_df = sorted_df.replace('TBD', None)

In [ ]:
sorted_df['Projection'] = pd.to_numeric(sorted_df.Projection)
sorted_df['Diff'] = pd.to_numeric(sorted_df.Diff)
sorted_df['Result'] = pd.to_numeric(sorted_df.Result)
sorted_df['Pick'] = pd.to_numeric(sorted_df.Pick)

In [ ]:
# Determine if website pick was a win or loss (for fun, potential spinoff angle).
conditions = [
    ((sorted_df['Result'] < sorted_df['Pick']) & (sorted_df['over_under'] == 'u')),
    ((sorted_df['Result'] < sorted_df['Pick']) & (sorted_df['over_under'] == 'o')),
    ((sorted_df['Result'] > sorted_df['Pick']) & (sorted_df['over_under'] =='o')),
    ((sorted_df['Result'] > sorted_df['Pick']) & (sorted_df['over_under'] == 'u')),
    (sorted_df['Result'].isna() == True)
    ]

# create a list of the values we want to assign for each condition
values = ['1', '0', '1', '0', np.nan]

# create a new column and use np.select to assign values to it using our lists as arguments
sorted_df['successful_pick'] = np.select(conditions, values)